In [ ]:
from scipy import io
from func import pca
import numpy as np
import matplotlib.pyplot as plt
x=io.loadmat('Yale_64x64.mat')
ins_perclass,class_number,train_test_split = 11,15,9
input_dim=x['fea'].shape[1]
feat=x['fea'].reshape(-1,ins_perclass,input_dim)
label=x['gnd'].reshape(-1,ins_perclass)

train_data,test_data = feat[:,:train_test_split,:].reshape(-1,input_dim),feat[:,train_test_split:,:].reshape(-1,input_dim)
train_label,test_label = label[:,:train_test_split].reshape(-1),label[:,train_test_split:].reshape(-1)

"""
train_data和test_data的维度 = (num_samples, num_features)
train_label和test_label的维度 = (num_samples, 1)
"""

In [ ]:


def lda(X, y, n_components):
    """
    LDA算法实现

    参数:
    X: numpy.ndarray, 输入数据，形状为 (样本数, 特征数)
    y: numpy.ndarray, 类别标签，形状为 (样本数, 1)
    n_components: int, 需要保留的投影维度

    返回:
    X_lda: numpy.ndarray, 降维后的数据
    components: numpy.ndarray, 投影矩阵
    """
    # 获取类别
    class_labels = np.unique(y)
    # 计算总体均值
    mean_overall = np.mean(X, axis=0)

    # 初始化类内散度矩阵和类间散度矩阵
    S_W = np.zeros((X.shape[1], X.shape[1]))
    S_B = np.zeros((X.shape[1], X.shape[1]))

    for c in class_labels:
        # 获取属于当前类别的样本
        X_c = X[y == c]
        # 计算当前类别的均值向量
        mean_c = np.mean(X_c, axis=0)
        # 计算类内散度矩阵
        S_W += np.dot((X_c - mean_c).T, (X_c - mean_c))
        # 计算类间散度矩阵
        n_c = X_c.shape[0]
        mean_diff = (mean_c - mean_overall).reshape(-1, 1)
        S_B += n_c * np.dot(mean_diff, mean_diff.T)

    # 计算 S_W^-1 * S_B
    A = np.linalg.pinv(S_W).dot(S_B)

    # 对矩阵 A 进行特征值分解
    eigenvalues, eigenvectors = np.linalg.eigh(A)

    # 按特征值从大到小排序
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors = eigenvectors[:, sorted_indices]

    # 选择前 n_components 个特征向量
    components = eigenvectors[:, :n_components]

    # 将数据投影到新空间
    X_lda = np.dot(X, components)

    return X_lda, components

In [ ]:
# 进行LDA降维，保留前8个主成分
X_pca, eigVects = lda(train_data, train_label, 12)

In [ ]:
idx = [1, 3, 11, 4, 5, 7, 8, 9]
# Plot the first 8 components as 64x64 images
plt.figure(figsize=(12, 6))
for i in range(8):
    component_image = np.real(eigVects[:, idx[i]]).reshape(64, 64)
    plt.subplot(2, 4, i + 1)
    plt.imshow(np.rot90(component_image,-1), cmap='gray')
    plt.title(f'Component {i + 1}')
    plt.axis('off')

plt.show()

In [ ]:
eigVects_top2 = eigVects[:, :2]
X_lda = np.dot(train_data, eigVects_top2)
# 绘图
from matplotlib.colors import ListedColormap
cmap = plt.get_cmap('tab20', class_number)

# 可视化降维后的训练和测试数据
def plot_lda_2d(X, labels, title):
    plt.figure(figsize=(10, 6))
    c=0;
    for label in np.unique(labels):
        plt.scatter(X[labels == label, 0], X[labels == label, 1], label=f'Class {label}', color=cmap(c))
        c = c+1
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.title(title)
    plt.legend()
    plt.show()

plot_lda_2d(X_lda, train_label, 'LDA of Training Data')

In [ ]:
X_lda_test = np.dot(test_data, eigVects_top2)
# 绘图
from matplotlib.colors import ListedColormap
cmap = plt.get_cmap('tab20', class_number)

# 可视化降维后的训练和测试数据
def plot_lda_2d(X, labels, title):
    plt.figure(figsize=(10, 6))
    c=0;
    for label in np.unique(labels):
        plt.scatter(X[labels == label, 0], X[labels == label, 1], label=f'Class {label}', color=cmap(c))
        c = c+1
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.title(title)
    plt.legend()
    plt.show()

plot_lda_2d(X_lda_test, test_label, 'PCA of Training Data')

In [ ]:
plt.imshow(np.rot90(np.real(eigVects[:, 0]+50).reshape(64, 64), -1), cmap='gray')
plt.show